<a href="https://colab.research.google.com/github/rena-tech/for-copy/blob/main/Station1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

| Version | Published Date| Details |
| -- | -- | -- |
| ver.1.0.0 | 2023/8/29 | 初版 |
| ver.1.0.1 | 2023/9/01 | 問題に補足を追記 |

# 正規表現について学ぼう

Python自動化Stationへようこそ。入門編に引き続き，Pythonを使った自動化を学んでいきましょう。このStationでは強力なツールである **正規表現** について学習します。正規表現はPythonに限らず，ほとんどすべてのプログラミングやExcelでも利用できる非常に重要なツールです。見慣れない表記が多くなりますが，しっかりと習得しましょう。

# 正規表現とパターンマッチ

ブラウザやテキストエディタで `Ctrl/Command - F` を押してテキスト検索を行ったことがある方は多いことでしょう。このStationで学ぶ **正規表現** では，さらに一歩推し進めて文字列を検索する「パターン」を指定します。

たとえば電話番号について考えます。正確な電話番号を知らなくても，みなさんなら `03-1234-5678` が電話番号であり `123,456,789` がそうではないことはよくおわかりでしょう。その他にも，メールアドレスは必ず `@` を含みますし，ウェブサイトのURLには `.` や `/` があります。

正規表現はこうしたパターンを記述するのに非常に便利です。しかしMS OfficeやExcelでも正規表現を使って文字列を検索したり置換したりできることを知っている人はそれほど多くはありません。正規表現を使えば，ソフトを使う人やプログラマーにとっても多くの時間を節約できます。 *プログラミングを学ぶ前に正規表現を学ぶべきだ* と主張する人さえいます。

このStationは以下のように進みます。

- 正規表現を使わずにテキストのパターンを検索するプログラムを書く
- 正規表現を使ってコードをコンパクトにする
- 正規表現のマッチングについて説明
- より強力な正規表現の機能を説明
- テキストから電話番号とメールアドレスを抽出する

## 正規表現を使わないテキストパターン検索

まずは文字列から電話番号を検索することを考えます。実際には電話番号にはより多くのパターンがありますが，簡単のために電話番号が 4桁の数字，ハイフン( `-` )，2桁の数字，ハイフン( `-` )，4桁の数字だとします。たとえば `0123-44-5678` がこのパターンに当てはまります。

それでは文字列がこのパターンに当てはまるかどうか調べ， `True` か `False` を返す関数 `is_phone_number()` を作りましょう。

In [ ]:
def is_phone_number(text):
    if len(text) != 12:
        return False
    for i in range(0, 4):
        if not text[i].isdecimal():
            return False
        if text[4] != '-':
            return False
        for i in range(5, 7):
            if not text[i].isdecimal():
                return False
        if text[7] != '-':
            return False
        for i in range(8, 12):
            if not text[i].isdecimal():
                return False
    return True

In [ ]:
# 正しい形式の電話番号
is_phone_number("0123-45-5678")

In [ ]:
# 正しくない形式の電話番号
is_phone_number("0-1233467777")

In [ ]:
# 正しくない形式
is_phone_number("I have an apple.")

`is_phone_number()` 関数は，文字列が正しい電話番号であるかどうかを何回かに分けて調べています。いずれかにおいて一致しなければ関数は `False` を返します。このプログラムは以下のように文字列を判定します。

- 文字列の長さがぴったり12文字かどうか
- 市外局番 (最初の4桁) が数字だけから構成されているかどうか
- 市外局番のあとが `-` になっているかどうか
- 2桁の数字が続くか
- もう一度 `-` が続くか
- 最後に4桁の数字が続くか

をすべて調べ，一致すれば `True` を返します。

`is_phone_number()` に `"0123-45-5678"` を渡すと `True` を返します。`"I have an apple."` を渡すと，最初のテストに落ちて `False` を返します。

長いテキストの中から電話番号のパターンを見つけるためには，さらにコードの追加が必要です。

In [ ]:
message = '今日の夜に0123-45-6789に電話してください。明日は休日なので0345-67-8910に電話してください。'

for i, _ in enumerate(message):
    chunk = message[i:i + 12]
    if is_phone_number(chunk):
        print(f'電話番号が見つかりました: {chunk}')

print("完了")

この関数は `for` ループを繰り返すたびに `message` から12文字のまとまりを切り出して変数 `chunk` に格納します。最初の繰り返しでは `i` は `0` なので `chunk` には `message[0:12]` (つまり `"今日の夜に0123-45"` ) が代入されます。その次では `i` は `1` になり `chunk` には `message[0:12]` (つまり `"日の夜に0123-45-"` ) という文字列が代入されます。つまり `for` ループのそれぞれの繰り返しにおいて `chunk` は

- `今日の夜に0123-45`
- `日の夜に0123-45-`
- `の夜に0123-45-6`
- `夜に0123-45-67`
- `に0123-45-678`
- `0123-45-6789`
- `123-45-6789に`
- `23-45-6789に電`
- …

のようになります。その `chunk` を `is_phone_number()` に渡して，電話番号のパターンに一致するか調べます。もし一致するのであれば `chunk` を表示します。

そうして `message` 全体をループし，12文字を切り出した `chunk` が電話番号かどうか調べて `is_phone_number()` が `True` なら `chunk` を表示します。メッセージ全体を処理したら「完了」を表示します。

この例では `message` の文字列は短いですし，何百万文字の長さになってもプログラム自体は1秒もあれば実行完了するでしょう。正規表現を使って電話番号を検索するプログラムも1秒以内に実行完了しますが，正規表現を使うとこういったプログラムをもっと簡潔に書けます。

---

ちなみに今回のコードで使っている [`enumerate()` 関数](https://docs.python.org/ja/3/library/functions.html#enumerate) はイテレータと呼ばれる繰り返しオブジェクトについて添字とそれ自身を返す組み込み関数です。また今回のコードではそれ自身は必要ありません。そのため不必要な変数であることを示すために `_` に代入しています。

## 正規表現を用いてテキストパターンを検索する

先に書いた電話番号検索プログラムはきちんと動作しますが，コードが長い割にはできることが限られています。 `is_phone_number()` 関数は17行ありますが，電話番号のパターンを1つだけしか検索できません。

もし電話番号が `123.444.5555` や `(0123)444-555` のような形式だったら対応できません。もし `+81-80-4444-9999` のように国番号がついていたらどうでしょうか。 `is_phone_number()` 関数はこのような形式を検証できません。こういったパターンのためのコードを追加すると，どんどんコードが長くなってしまいます。

**正規表現 (Regular Expression, 縮めて regex)** は，テキストのパターンの記述方法です。たとえば正規表現で `\d` は数字1文字を表します。 `d` は `decimal` の頭文字です。正規表現では `\d\d\d\d-\d\d-\d\d\d\d` は先の `is_phone_number()` 関数と同じ文字列 (4桁の数字，ハイフン，2桁の数字，ハイフン，4桁の数字) にマッチします。その他の文字列は `\d\d\d\d-\d\d-\d\d\d\d` にはマッチしません。

正規表現それ自身はもう少し簡潔に書けます。たとえばパターンの後 `{}` に3を入れて書いた `{3}` は「直前のパターンと3回マッチする」ことを意味します。したがって，正しい番号にマッチする正規表現は `\d{4}-\d{2}-\d{4}` と少し短く書けます。

## Regexオブジェクトを生成する

Pythonでの正規表現は，すべて `re` モジュールの中にあります。そのためPythonで正規表現を使うときはまず

In [ ]:
import re

と入力します。 `re.compile()` に正規表現パターンを表す文字列を渡すとRegexパターンオブジェクト (簡単にRegexオブジェクトとも言います)が返ってきます。

電話番号パターンにマッチする Regex オブジェクトを生成するにはこのように入力します。

In [ ]:
phone_num_regex = re.compile(r'\d\d-\d\d\d\d-\d\d\d\d')

`\d` は1文字の数字であり `\d\d-\d\d\d\d-\d\d\d\d` は正しい電話番号のパターンを表す正規表現です。 `\` はPythonでは特別な意味を持つため `""` や `''` と一緒に使うには [エスケープ](https://docs.python.org/ja/3.9/reference/lexical_analysis.html#string-and-bytes-literals) が必要です。これをいちいち行うのは面倒なため raw 文字列を用いて `''` の前に `r` を配置すると便利です。

これで，変数 `phone_num_regex` にはRegexオブジェクトが格納されました。

## Regexオブジェクトとマッチする

Regexオブジェクトの `search()` メソッドは，渡された文字列の中から正規表現とマッチする部分を探します。見つからなければ `None` を返し，見つかれば `Match` オブジェクトを返します。 `Match` オブジェクトには `group()` メソッドがあり，実際にマッチしたテキストを返します。

In [ ]:
phone_num_regex = re.compile(r'\d\d\d\d-\d\d-\d\d\d\d')
mo = phone_num_regex.search('お店の電話番号は0123-45-6789です。')
print(f'電話番号が見つかりました: {mo.group()}')

変数名 `mo` は `Match` オブジェクトに用いられる汎用名です。この例は最初は複雑に見えるかもしれませんが，先のプログラムよりはるかに短いのにも関わらず同じ働きをします。

ここでは，パターンを `re.compile()` に渡し，返り値の `Regex` オブジェクトを `phone_num_regex` に格納します。次にそれに対して `search()` を呼び出し，検索対象の文字列を渡します。検索結果は変数 `mo` に格納されます。この例では検索結果が見つかるため `None` ではなく `Match` オブジェクトが返ります。 `mo` に対して `group()` を呼ぶことでマッチした部分を返しています。

In [ ]:
mo = re.search(r'\d\d\d\d-\d\d-\d\d\d\d', 'お店の電話番号は0123-45-6789です。')
print(f'電話番号が見つかりました: {mo.group()}')

このように `compile` を行わず `re.search()` メソッドに直接 `Regex` オブジェクトを渡すこともできます。しかし `compile` する場合に比べて実行時間が遅くなることが多いです。

Google Colabでは `%%timeit` をセルの一番上に書くとそのセルが自動的に何度も実行され，実行時間を計測できます。実際に試してみましょう。

In [ ]:
%%timeit
mo = re.search(r'\d\d\d\d-\d\d-\d\d\d\d', 'お店の電話番号は0123-45-6789です。')

In [ ]:
%%timeit
mo = phone_num_regex.search('お店の電話番号は0123-45-6789です。')

# 確認テスト

正規表現を使って以下の問題に答えてください。

- `012012` という文字列のみにマッチする正規表現パターンを書いてください。
  - 難しく考えず、極めて単純に考えてみてください。
- 日本の郵便番号にマッチする正規表現パターンを書いてください。ここで郵便番号とは `数字3桁` `-` `数字4ケタ` で表される文字列とします。
  - 数字には `\d` を利用してください。
  - `(0|1|2|3|4|5|6|7|8|9)` のような冗長な表現はいくつかありますが、冗長な表現は使用せず、簡潔に書くようにしてください。
  - また、本 Stationで紹介した手法を利用してください。Station2以降ではより簡潔に書ける方法を説明しますが、ここでは利用しないでください。

In [ ]:
# 確認テスト (1)
import re
mo = re.search(
    r''# <- ここに正規表現を書きます (ここに記入した内容を回答フォームに登録してください)
    , '電話番号:0120-012-012、郵便番号:123-4567、住所:東京都千代田区012012')
print(f'マッチした文字列が見つかりました: {mo.group()}')

In [ ]:
# 確認テスト (2)
import re
mo = re.search(
    r''# <- ここに正規表現を書きます (ここに記入した内容を回答フォームに登録してください)
    , '電話番号:0120-012-012、郵便番号:123-4567、住所:東京都千代田区012012')
print(f'マッチした郵便番号が見つかりました: {mo.group()}')